In [ ]:
# !pip install langchain langchain-community langchain-openai llama-index openai chromadb deepeval tenacity
# !pip install rouge_score
# !pip install nltk bert-score transformers
# !pip install sentence-transformers datasets
# !pip install cohere
# !pip install -U llama-index sentence-transformers
# !pip install llama-index-embeddings-langchain

In [10]:
import warnings
warnings.filterwarnings('ignore')
import json
import torch
import os
import re
import time
import shutil
from datetime import datetime
import csv
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import pandas as pd
import numpy as np
from typing import List, Tuple
from collections import defaultdict

import nltk
from nltk.translate.bleu_score import sentence_bleu
from nltk.tokenize import word_tokenize
from nltk.translate.meteor_score import meteor_score
from bert_score import score as bert_score
from transformers import pipeline
nltk.download('punkt_tab')
nltk.download('punkt')
nltk.download('wordnet')

import logging
from datasets import load_dataset, concatenate_datasets, Dataset, DatasetDict
from rouge_score import rouge_scorer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sentence_transformers.evaluation import InformationRetrievalEvaluator, SequentialEvaluator
from sentence_transformers.util import cos_sim
from sentence_transformers.losses import MatryoshkaLoss, MultipleNegativesRankingLoss, CosineSimilarityLoss, TripletLoss
from sentence_transformers import SentenceTransformerTrainingArguments, SentenceTransformerTrainer, SentenceTransformer, util, LoggingHandler, losses, InputExample
from sentence_transformers.training_args import BatchSamplers
from transformers import EarlyStoppingCallback, AdamW, get_linear_schedule_with_warmup

from langchain.vectorstores import Chroma
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from llama_index.core.node_parser import SimpleNodeParser
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.embeddings.langchain import LangchainEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.llms.openai import OpenAI as LlamaOpenAI
from llama_index.core import Document as LlamaDocument, StorageContext, load_index_from_storage, ServiceContext
from llama_index.core.prompts import PromptTemplate
from llama_index.core.response_synthesizers import get_response_synthesizer
from llama_index.core.schema import MetadataMode

from huggingface_hub import login
from transformers import AutoModel, AutoTokenizer
from huggingface_hub import upload_file

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /home/jupyter/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jupyter/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### 1. Load the Sample Query Dataset

In [2]:
# Define file paths

# Load datasets
def load_data(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

train_data = load_data("train_test_df/train_dataset.json")
val_data = load_data("train_test_df/val_dataset.json")

# Prepare datasets
def prepare_dataset(data):
    queries = data['queries']
    corpus = data['corpus']
    relevant_docs = data['relevant_docs']

    dataset = []
    for query_id, query in queries.items():
        doc_ids = relevant_docs[query_id]
        for doc_id in doc_ids:
            dataset.append({
                "id": query_id,
                "anchor": query,
                "positive": corpus[doc_id]
            })
    return dataset

train_list = prepare_dataset(train_data)
val_list = prepare_dataset(val_data)

# Combine all data
all_data = train_list + val_list

train_val, test = train_test_split(all_data, test_size=0.15, random_state=42)
train, val = train_test_split(train_val, test_size=0.1765, random_state=42)

# Create datasets
train_dataset = Dataset.from_list(train)
val_dataset = Dataset.from_list(val)
test_dataset = Dataset.from_list(test)

# Combine datasets
dataset = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

print(f"Train size: {len(dataset['train'])}")
print(f"Validation size: {len(dataset['validation'])}")
print(f"Test size: {len(dataset['test'])}")

Train size: 475
Validation size: 103
Test size: 102


In [3]:
# Prepare evaluation data
corpus = dict(zip(dataset["test"]["id"], dataset["test"]["positive"]))
queries = dict(zip(dataset["test"]["id"], dataset["test"]["anchor"]))
relevant_docs = {q_id: [q_id] for q_id in queries}

In [4]:
# Enhanced Data Augmentation function
def augment_data(example):
    augmented = example.copy()
    # Randomly swap anchor and positive with higher probability
    if random.random() < 0.8:
        augmented["anchor"], augmented["positive"] = augmented["positive"], augmented["anchor"]
    
    # Add more complex noise or paraphrases to embeddings (simulated here)
    if random.random() < 0.6:  # Increased probability of augmentation
        words = augmented["anchor"].split()
        random.shuffle(words)
        augmented["anchor"] += " " + " ".join(words[:min(5, len(words))])  # More aggressive shuffling
    
    return augmented

# Apply data augmentation
augmented_train = Dataset.from_list([augment_data(ex) for ex in train])
dataset["train"] = augmented_train

In [5]:
class RegularizedTransformer(SentenceTransformer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        # Increase model capacity and regularization
        for layer in self.modules():
            if isinstance(layer, torch.nn.Linear):
                # Higher dropout for better regularization
                layer.dropout = torch.nn.Dropout(0.5)
                # Add layer normalization
                layer.layer_norm = torch.nn.LayerNorm(layer.out_features)
            if hasattr(layer, 'dropout'):
                layer.dropout.p = 0.5

In [6]:
# Model setup
model_id = "BAAI/bge-large-en-v1.5"
matryoshka_dimensions = [1024, 768, 512, 256]  # Adjusted dimensions for the new model
# device="cuda" if torch.cuda.is_available() else "cpu",
model = RegularizedTransformer(
    model_id,
    device= torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    trust_remote_code=True
)

# Create evaluators
matryoshka_evaluators = []
for dim in matryoshka_dimensions:
    ir_evaluator = InformationRetrievalEvaluator(
        queries=queries,
        corpus=corpus,
        relevant_docs=relevant_docs,
        name=f"dim_{dim}",
        truncate_dim=dim,
        score_functions={"cosine": cos_sim},
    )
    matryoshka_evaluators.append(ir_evaluator)

evaluator = SequentialEvaluator(matryoshka_evaluators)

In [7]:
print("Evaluation results before fine-tuning:")
results = evaluator(model)
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print(f"{key}: {results[key]}")

Evaluation results before fine-tuning:
dim_1024_cosine_ndcg@10: 0.516968527386548
dim_768_cosine_ndcg@10: 0.5254591166761731
dim_512_cosine_ndcg@10: 0.5247580677304011
dim_256_cosine_ndcg@10: 0.5305440710647368


### 2. Fine-Tune

In [7]:
# Custom callback for logging metrics and losses
class MetricsLogger(EarlyStoppingCallback):
    def __init__(self, logger=None, patience=5, threshold=0.005):  # Reduced patience and threshold for earlier stopping
        super().__init__(early_stopping_patience=patience, early_stopping_threshold=threshold)
        self.logger = logger or logging.getLogger(__name__)
        self.best_metric = None
        self.train_losses = []
        self.val_losses = []

    def on_evaluate(self, args, state, control, metrics, **kwargs):
        self.logger.info("\nEvaluation metrics:")
        for key, value in metrics.items():
            self.logger.info(f"{key}: {value}")

        self.train_losses.append(metrics.get("train_loss", 0))
        self.val_losses.append(metrics.get("eval_loss", 0))

        current_metric = metrics.get("eval_dim_1024_cosine_ndcg@10")
        if current_metric is not None:
            if self.best_metric is None or current_metric > self.best_metric:
                self.best_metric = current_metric
                self.logger.info(f"New best metric: {current_metric}")

        super().on_evaluate(args, state, control, metrics, **kwargs)

In [8]:
# Function to clean up previous model files
def cleanup_model_directory(directory):
    """Remove all files in the specified directory"""
    if os.path.exists(directory):
        shutil.rmtree(directory)
        print(f"Cleaned up directory: {directory}")
    os.makedirs(directory)
    print(f"Created new directory: {directory}")

# Setup logging
def setup_logging(output_dir):
    """Setup logging configuration"""
    # Create logs directory if it doesn't exist
    logs_dir = os.path.join(output_dir, 'logs')
    os.makedirs(logs_dir, exist_ok=True)

    # Create timestamp for unique log file
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    log_file = os.path.join(logs_dir, f'training_log_{timestamp}.txt')

    # Configure logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file),
            logging.StreamHandler(),
            LoggingHandler()
        ]
    )

    return logging.getLogger(__name__)

In [9]:
# Main execution for post-fine-tuning
if __name__ == "__main__":
    post_finetune_output_dir = "bge-large-post-finetuned-edit"
    os.makedirs(post_finetune_output_dir, exist_ok=True)
    
    logger = setup_logging(post_finetune_output_dir)
    logger.info("Starting post-fine-tune training")

    train_loss = losses.MultipleNegativesRankingLoss(model)

    # Training arguments
    args = SentenceTransformerTrainingArguments(
        output_dir=post_finetune_output_dir,
        num_train_epochs=200,
        per_device_train_batch_size=32,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=64,
        warmup_ratio=0.1,
        learning_rate=2e-5,
        weight_decay=0.01,
        lr_scheduler_type="cosine_with_restarts",
        optim="adamw_torch",
        fp16=True,
        eval_strategy="steps",
        evaluation_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=50,
        logging_dir=os.path.join(post_finetune_output_dir, 'tensorboard'),
        logging_strategy="steps",
        logging_steps=10,
        save_total_limit=2,
        load_best_model_at_end=True,
        metric_for_best_model="eval_dim_1024_cosine_ndcg@10",
        greater_is_better=True,
        max_grad_norm=1.0,
        gradient_checkpointing=True
    )

    # Create metrics logger callback with adjusted parameters
    metrics_logger = MetricsLogger(
        logger=logger,
        patience=5,           # Reduced patience for earlier stopping
        threshold=0.0005      # Reduced threshold for more sensitive stopping
    )

    # Create trainer with the metrics logger
    trainer = SentenceTransformerTrainer(
        model=model,
        args=args,
        train_dataset=dataset["train"].select_columns(["positive", "anchor"]),
        eval_dataset=dataset["validation"].select_columns(["positive", "anchor"]),
        loss=train_loss,
        evaluator=evaluator,
        callbacks=[metrics_logger]  # Use the metrics logger here
    )

    # Training
    try:
        trainer.train()
        logger.info("Fine-tuning completed successfully")
    except Exception as e:
        logger.error(f"Fine-tuning failed with error: {str(e)}")
        raise

    # Save the model
    try:
        trainer.save_model(post_finetune_output_dir)
        logger.info(f"Model saved to {post_finetune_output_dir}")
    except Exception as e:
        logger.error(f"Failed to save model: {str(e)}")
        raise

    # Final evaluation
    logger.info("Running final evaluation...")
    fine_tuned_model = SentenceTransformer(post_finetune_output_dir)
    results = evaluator(fine_tuned_model)

    # Log final results
    logger.info("\nFinal evaluation results:")
    for dim in matryoshka_dimensions:
        key = f"dim_{dim}_cosine_ndcg@10"
        logger.info(f"{key}: {results[key]}")

    logger.info("Training pipeline completed")

2024-11-10 15:02:17,460 - INFO - Starting post-fine-tune training


2024-11-10 15:02:17,460 - INFO - Starting post-fine-tune training


2024-11-10 15:02:19,358 - WARNING - Column 'anchor' is at index 1, whereas a column with this name is usually expected at index 0. Note that the column order can be important for some losses, e.g. MultipleNegativesRankingLoss will always consider the first column as the anchor and the second as the positive, regardless of the dataset column names. Consider renaming the columns to match the expected order, e.g.:
dataset = dataset.select_columns(['anchor', 'positive', 'negative'])


2024-11-10 15:02:19,358 - WARNING - Column 'anchor' is at index 1, whereas a column with this name is usually expected at index 0. Note that the column order can be important for some losses, e.g. MultipleNegativesRankingLoss will always consider the first column as the anchor and the second as the positive, regardless of the dataset column names. Consider renaming the columns to match the expected order, e.g.:
dataset = dataset.select_columns(['anchor', 'positive', 'negative'])


Step,Training Loss,Validation Loss,Dim 1024 Cosine Accuracy@1,Dim 1024 Cosine Accuracy@3,Dim 1024 Cosine Accuracy@5,Dim 1024 Cosine Accuracy@10,Dim 1024 Cosine Precision@1,Dim 1024 Cosine Precision@3,Dim 1024 Cosine Precision@5,Dim 1024 Cosine Precision@10,Dim 1024 Cosine Recall@1,Dim 1024 Cosine Recall@3,Dim 1024 Cosine Recall@5,Dim 1024 Cosine Recall@10,Dim 1024 Cosine Ndcg@10,Dim 1024 Cosine Mrr@10,Dim 1024 Cosine Map@100,Dim 768 Cosine Accuracy@1,Dim 768 Cosine Accuracy@3,Dim 768 Cosine Accuracy@5,Dim 768 Cosine Accuracy@10,Dim 768 Cosine Precision@1,Dim 768 Cosine Precision@3,Dim 768 Cosine Precision@5,Dim 768 Cosine Precision@10,Dim 768 Cosine Recall@1,Dim 768 Cosine Recall@3,Dim 768 Cosine Recall@5,Dim 768 Cosine Recall@10,Dim 768 Cosine Ndcg@10,Dim 768 Cosine Mrr@10,Dim 768 Cosine Map@100,Dim 512 Cosine Accuracy@1,Dim 512 Cosine Accuracy@3,Dim 512 Cosine Accuracy@5,Dim 512 Cosine Accuracy@10,Dim 512 Cosine Precision@1,Dim 512 Cosine Precision@3,Dim 512 Cosine Precision@5,Dim 512 Cosine Precision@10,Dim 512 Cosine Recall@1,Dim 512 Cosine Recall@3,Dim 512 Cosine Recall@5,Dim 512 Cosine Recall@10,Dim 512 Cosine Ndcg@10,Dim 512 Cosine Mrr@10,Dim 512 Cosine Map@100,Dim 256 Cosine Accuracy@1,Dim 256 Cosine Accuracy@3,Dim 256 Cosine Accuracy@5,Dim 256 Cosine Accuracy@10,Dim 256 Cosine Precision@1,Dim 256 Cosine Precision@3,Dim 256 Cosine Precision@5,Dim 256 Cosine Precision@10,Dim 256 Cosine Recall@1,Dim 256 Cosine Recall@3,Dim 256 Cosine Recall@5,Dim 256 Cosine Recall@10,Dim 256 Cosine Ndcg@10,Dim 256 Cosine Mrr@10,Dim 256 Cosine Map@100,Sequential Score
50,3.003500,2.770679,0.333333,0.568627,0.647059,0.774510,0.333333,0.189542,0.129412,0.077451,0.333333,0.568627,0.647059,0.774510,0.538142,0.463901,0.473468,0.352941,0.558824,0.637255,0.754902,0.352941,0.186275,0.127451,0.075490,0.352941,0.558824,0.637255,0.754902,0.537784,0.469592,0.480610,0.323529,0.539216,0.637255,0.764706,0.323529,0.179739,0.127451,0.076471,0.323529,0.539216,0.637255,0.764706,0.525857,0.451280,0.461131,0.323529,0.568627,0.647059,0.774510,0.323529,0.189542,0.129412,0.077451,0.323529,0.568627,0.647059,0.774510,0.534415,0.458750,0.468974,0.468974
100,2.149100,2.548931,0.323529,0.558824,0.705882,0.803922,0.323529,0.186275,0.141176,0.080392,0.323529,0.558824,0.705882,0.803922,0.548046,0.467137,0.479392,0.323529,0.558824,0.715686,0.823529,0.323529,0.186275,0.143137,0.082353,0.323529,0.558824,0.715686,0.823529,0.552447,0.467153,0.476950,0.323529,0.568627,0.715686,0.833333,0.323529,0.189542,0.143137,0.083333,0.323529,0.568627,0.715686,0.833333,0.557649,0.471047,0.480542,0.323529,0.539216,0.686275,0.852941,0.323529,0.179739,0.137255,0.085294,0.323529,0.539216,0.686275,0.852941,0.556687,0.465219,0.472693,0.472693
150,1.590800,2.570700,0.303922,0.588235,0.735294,0.862745,0.303922,0.196078,0.147059,0.086275,0.303922,0.588235,0.735294,0.862745,0.565426,0.471250,0.479961,0.303922,0.598039,0.715686,0.852941,0.303922,0.199346,0.143137,0.085294,0.303922,0.598039,0.715686,0.852941,0.560124,0.467480,0.476833,0.313725,0.578431,0.725490,0.882353,0.313725,0.192810,0.145098,0.088235,0.313725,0.578431,0.725490,0.882353,0.569482,0.471876,0.478619,0.323529,0.627451,0.725490,0.862745,0.323529,0.209150,0.145098,0.086275,0.323529,0.627451,0.725490,0.862745,0.579334,0.489616,0.497761,0.497761
200,1.236600,2.621908,0.284314,0.607843,0.735294,0.901961,0.284314,0.202614,0.147059,0.090196,0.284314,0.607843,0.735294,0.901961,0.574014,0.470538,0.476309,0.284314,0.598039,0.735294,0.892157,0.284314,0.199346,0.147059,0.089216,0.284314,0.598039,0.735294,0.892157,0.568665,0.466577,0.473080,0.274510,0.627451,0.764706,0.882353,0.274510,0.209150,0.152941,0.088235,0.274510,0.627451,0.764706,0.882353,0.567944,0.467628,0.474978,0.274510,0.607843,0.754902,0.892157,0.274510,0.202614,0.150980,0.089216,0.274510,0.607843,0.754902,0.892157,0.566946,0.463951,0.470186,0.470186
250,1.056300,2.631725,0.284314,0.617647,0.754902,0.901961,0.284314,0.205882,0.150980,0.090196,0.284314,0.617647,0.754902,0.901961,0.57

...

2024-11-10 17:38:00,905 - INFO - Queries: 102
2024-11-10 17:38:00,906 - INFO - Corpus: 102

2024-11-10 17:38:00,912 - INFO - Score-Function: cosine
2024-11-10 17:38:00,913 - INFO - Accuracy@1: 33.33%
2024-11-10 17:38:00,913 - INFO - Accuracy@3: 64.71%
2024-11-10 17:38:00,914 - INFO - Accuracy@5: 76.47%
2024-11-10 17:38:00,915 - INFO - Accuracy@10: 93.14%
2024-11-10 17:38:00,915 - INFO - Precision@1: 33.33%
2024-11-10 17:38:00,916 - INFO - Precision@3: 21.57%
2024-11-10 17:38:00,916 - INFO - Precision@5: 15.29%
2024-11-10 17:38:00,917 - INFO - Precision@10: 9.31%
2024-11-10 17:38:00,918 - INFO - Recall@1: 33.33%
2024-11-10 17:38:00,918 - INFO - Recall@3: 64.71%
2024-11-10 17:38:00,919 - INFO - Recall@5: 76.47%
2024-11-10 17:38:00,919 - INFO - Recall@10: 93.14%
2024-11-10 17:38:00,920 - INFO - MRR@10: 0.5177
2024-11-10 17:38:00,920 - INFO - NDCG@10: 0.6169
2024-11-10 17:38:00,921 - INFO - MAP@100: 0.5209
2024-11-10 17:38:00,922 - INFO - Information Retrieval Evaluation of the model on th

2024-11-10 17:38:11,479 - INFO - Queries: 102
2024-11-10 17:38:11,480 - INFO - Corpus: 102

2024-11-10 17:38:11,486 - INFO - Score-Function: cosine
2024-11-10 17:38:11,487 - INFO - Accuracy@1: 32.35%
2024-11-10 17:38:11,488 - INFO - Accuracy@3: 65.69%
2024-11-10 17:38:11,488 - INFO - Accuracy@5: 78.43%
2024-11-10 17:38:11,490 - INFO - Accuracy@10: 93.14%
2024-11-10 17:38:11,490 - INFO - Precision@1: 32.35%
2024-11-10 17:38:11,491 - INFO - Precision@3: 21.90%
2024-11-10 17:38:11,492 - INFO - Precision@5: 15.69%
2024-11-10 17:38:11,493 - INFO - Precision@10: 9.31%
2024-11-10 17:38:11,494 - INFO - Recall@1: 32.35%
2024-11-10 17:38:11,495 - INFO - Recall@3: 65.69%
2024-11-10 17:38:11,496 - INFO - Recall@5: 78.43%
2024-11-10 17:38:11,497 - INFO - Recall@10: 93.14%
2024-11-10 17:38:11,498 - INFO - MRR@10: 0.5124
2024-11-10 17:38:11,499 - INFO - NDCG@10: 0.6131
2024-11-10 17:38:11,499 - INFO - MAP@100: 0.5155
2024-11-10 17:38:11,501 - INFO - Information Retrieval Evaluation of the model on th

2024-11-10 17:38:11,479 - INFO - Queries: 102
2024-11-10 17:38:11,480 - INFO - Corpus: 102

2024-11-10 17:38:11,486 - INFO - Score-Function: cosine
2024-11-10 17:38:11,487 - INFO - Accuracy@1: 32.35%
2024-11-10 17:38:11,488 - INFO - Accuracy@3: 65.69%
2024-11-10 17:38:11,488 - INFO - Accuracy@5: 78.43%
2024-11-10 17:38:11,490 - INFO - Accuracy@10: 93.14%
2024-11-10 17:38:11,490 - INFO - Precision@1: 32.35%
2024-11-10 17:38:11,491 - INFO - Precision@3: 21.90%
2024-11-10 17:38:11,492 - INFO - Precision@5: 15.69%
2024-11-10 17:38:11,493 - INFO - Precision@10: 9.31%
2024-11-10 17:38:11,494 - INFO - Recall@1: 32.35%
2024-11-10 17:38:11,495 - INFO - Recall@3: 65.69%
2024-11-10 17:38:11,496 - INFO - Recall@5: 78.43%
2024-11-10 17:38:11,497 - INFO - Recall@10: 93.14%
2024-11-10 17:38:11,498 - INFO - MRR@10: 0.5124
2024-11-10 17:38:11,499 - INFO - NDCG@10: 0.6131
2024-11-10 17:38:11,499 - INFO - MAP@100: 0.5155
2024-11-10 17:38:11,501 - INFO - Information Retrieval Evaluation of the model on th

2024-11-10 17:38:21,964 - INFO - Queries: 102
2024-11-10 17:38:21,965 - INFO - Corpus: 102

2024-11-10 17:38:21,971 - INFO - Score-Function: cosine
2024-11-10 17:38:21,971 - INFO - Accuracy@1: 30.39%
2024-11-10 17:38:21,972 - INFO - Accuracy@3: 61.76%
2024-11-10 17:38:21,972 - INFO - Accuracy@5: 80.39%
2024-11-10 17:38:21,973 - INFO - Accuracy@10: 92.16%
2024-11-10 17:38:21,973 - INFO - Precision@1: 30.39%
2024-11-10 17:38:21,974 - INFO - Precision@3: 20.59%
2024-11-10 17:38:21,975 - INFO - Precision@5: 16.08%
2024-11-10 17:38:21,976 - INFO - Precision@10: 9.22%
2024-11-10 17:38:21,976 - INFO - Recall@1: 30.39%
2024-11-10 17:38:21,977 - INFO - Recall@3: 61.76%
2024-11-10 17:38:21,977 - INFO - Recall@5: 80.39%
2024-11-10 17:38:21,978 - INFO - Recall@10: 92.16%
2024-11-10 17:38:21,978 - INFO - MRR@10: 0.4994
2024-11-10 17:38:21,979 - INFO - NDCG@10: 0.6013
2024-11-10 17:38:21,979 - INFO - MAP@100: 0.5032
2024-11-10 17:38:21,980 - INFO - 
Final evaluation results:
2024-11-10 17:38:21,981 

2024-11-10 17:38:21,964 - INFO - Queries: 102
2024-11-10 17:38:21,965 - INFO - Corpus: 102

2024-11-10 17:38:21,971 - INFO - Score-Function: cosine
2024-11-10 17:38:21,971 - INFO - Accuracy@1: 30.39%
2024-11-10 17:38:21,972 - INFO - Accuracy@3: 61.76%
2024-11-10 17:38:21,972 - INFO - Accuracy@5: 80.39%
2024-11-10 17:38:21,973 - INFO - Accuracy@10: 92.16%
2024-11-10 17:38:21,973 - INFO - Precision@1: 30.39%
2024-11-10 17:38:21,974 - INFO - Precision@3: 20.59%
2024-11-10 17:38:21,975 - INFO - Precision@5: 16.08%
2024-11-10 17:38:21,976 - INFO - Precision@10: 9.22%
2024-11-10 17:38:21,976 - INFO - Recall@1: 30.39%
2024-11-10 17:38:21,977 - INFO - Recall@3: 61.76%
2024-11-10 17:38:21,977 - INFO - Recall@5: 80.39%
2024-11-10 17:38:21,978 - INFO - Recall@10: 92.16%
2024-11-10 17:38:21,978 - INFO - MRR@10: 0.4994
2024-11-10 17:38:21,979 - INFO - NDCG@10: 0.6013
2024-11-10 17:38:21,979 - INFO - MAP@100: 0.5032
2024-11-10 17:38:21,980 - INFO - 
Final evaluation results:
2024-11-10 17:38:21,981 

In [ ]:
# Evaluate the fine-tuned model
fine_tuned_model = SentenceTransformer(
    output_dir, device="cuda" if torch.cuda.is_available() else "cpu"
)
results = evaluator(fine_tuned_model)

### 3. Evalution Metrics

In [ ]:
fine_tuned_model = SentenceTransformer(post_finetune_output_dir)
results = evaluator(fine_tuned_model)

In [10]:
# Print the main scores
print("Evaluation results after fine-tuning:")
for dim in matryoshka_dimensions:
    key = f"dim_{dim}_cosine_ndcg@10"
    print(f"{key}: {results[key]}")

Evaluation results after fine-tuning:
dim_1024_cosine_ndcg@10: 0.6179661058792598
dim_768_cosine_ndcg@10: 0.6169193146181351
dim_512_cosine_ndcg@10: 0.6130736364652425
dim_256_cosine_ndcg@10: 0.6013479428362282


### 4. Query Result

In [88]:
# Folder path and file names
folder_path = "scraped_tabs"
file_names = [
    "Capstone Projects.txt", "Career Outcomes.txt", "Course Progressions.txt",
    "Events & Deadlines.txt", "FAQs.txt", "Faculty, Instructors, Staff.txt",
    "How to Apply.txt", "In-Person Program.txt", "MS_ADS_Campus.txt",
    "Online Program.txt", "Our Students.txt", "Tuition_Fees_Aid.txt",
    "scraped_main_content.txt", "scraped_valid_links.txt"
]

# Construct full paths
txt_paths = [os.path.join(folder_path, file_name) for file_name in file_names]

def process_hyperlinks(text):
    # Regular expression to find URLs and their associated text
    url_pattern = r'\[(.*?)\]\((https?://[^\s\)]+)\)'
    return re.sub(url_pattern, r'[\1](\2)', text)

# Load documents
loaders = [TextLoader(path) for path in txt_paths]
documents = []
for loader in loaders:
    docs = loader.load()
    for doc in docs:
        doc.page_content = process_hyperlinks(doc.page_content)
    documents.extend(docs)

In [89]:
# Load the fine-tuned model
fine_tuned_model_path = "bge-large-post-finetuned-edit"
fine_tuned_model = SentenceTransformer(fine_tuned_model_path, device="cuda" if torch.cuda.is_available() else "cpu")

2024-11-10 21:11:07,047 - INFO - Load pretrained SentenceTransformer: bge-large-post-finetuned-edit
Some weights of the model checkpoint at bge-large-post-finetuned-edit were not used when initializing BertModel: ['encoder.layer.0.attention.output.dense.layer_norm.bias', 'encoder.layer.0.attention.output.dense.layer_norm.weight', 'encoder.layer.0.attention.self.key.layer_norm.bias', 'encoder.layer.0.attention.self.key.layer_norm.weight', 'encoder.layer.0.attention.self.query.layer_norm.bias', 'encoder.layer.0.attention.self.query.layer_norm.weight', 'encoder.layer.0.attention.self.value.layer_norm.bias', 'encoder.layer.0.attention.self.value.layer_norm.weight', 'encoder.layer.0.intermediate.dense.layer_norm.bias', 'encoder.layer.0.intermediate.dense.layer_norm.weight', 'encoder.layer.0.output.dense.layer_norm.bias', 'encoder.layer.0.output.dense.layer_norm.weight', 'encoder.layer.1.attention.output.dense.layer_norm.bias', 'encoder.layer.1.attention.output.dense.layer_norm.weight', 'enc

2024-11-10 21:11:07,047 - INFO - Load pretrained SentenceTransformer: bge-large-post-finetuned-edit


In [90]:
# Define embedding function using fine-tuned model
def get_fine_tuned_embedding(text):
    return fine_tuned_model.encode(text, convert_to_tensor=True).cpu().numpy()

In [91]:
# Set API keys
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

In [92]:
# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(documents)

In [93]:
# Define query prompt template
query_prompt_template = PromptTemplate(
    template=(
        "You are an AI assistant for the University of Chicago's Applied Data Science program. "
        "Your role is to provide accurate and concise information exactly as it appears in the source material. "
        "Follow these rules:\n"
        "1. Provide answers in the exact format and wording as shown in the source material\n"
        "2. When the source material contains URLs or webpage references, include them in markdown format [text](url)\n"
        "3. Keep responses concise and direct\n"
        "4. Maintain the exact formatting, including bullet points and line breaks\n"
        "5. If multiple sources contain the same information, use the most recent or most complete version\n\n"
        "Context: {context_str}\n\n"
        "Question: {query_str}\n\n"
        "Answer: "
    )
)

In [94]:
# Build or load index
def build_or_load_index(_documents, index_path="./saved_index"):
    if os.path.exists(index_path):
        storage_context = StorageContext.from_defaults(persist_dir=index_path)
        index = load_index_from_storage(storage_context)
    else:
        # Configure node parser to preserve formatting and links
        node_parser = SimpleNodeParser.from_defaults(
            chunk_size=1000,
            chunk_overlap=200,
            include_metadata=True
        )
        
        llama_documents = [LlamaDocument.from_langchain_format(doc) for doc in splits]
        
        llm = ChatOpenAI(
            temperature=0.0, 
            model_name="gpt-3.5-turbo",
            max_tokens=512
        )
        
        service_context = ServiceContext.from_defaults(
            llm=llm,
            embed_model=get_fine_tuned_embedding,
            node_parser=node_parser
        )
        
        index = VectorStoreIndex.from_documents(
            llama_documents,
            service_context=service_context
        )
        index.storage_context.persist(persist_dir=index_path)
    
    # Configure response synthesizer to maintain formatting
    response_synthesizer = get_response_synthesizer(
        response_mode="compact",
        text_qa_template=query_prompt_template
    )
    
    return index.as_query_engine(
        similarity_top_k=10,
        response_synthesizer=response_synthesizer,
        structured_answer_filtering=True
    )

llama_query_engine = build_or_load_index(documents)

2024-11-10 21:11:45,650 - INFO - Loading all indices.


2024-11-10 21:11:45,650 - INFO - Loading all indices.


In [95]:
def evaluate_rag_systems(query):
    start_time = time.time()
    llama_result = llama_query_engine.query(query)
    llama_time = time.time() - start_time

    print(f"Query: {query}\n")
    print(f"LlamaIndex Answer: {llama_result}")
    print(f"LlamaIndex Response Time: {llama_time:.2f} seconds\n")

    print("Evaluation:")

    scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
    llama_rouge = scorer.score(query, str(llama_result))
    rouge_score = llama_rouge['rougeL'].fmeasure
    print(f"1. ROUGE-L F1 Score: LlamaIndex ({rouge_score:.4f})")

    reference = [word_tokenize(query.lower())]
    meteor_score_value = meteor_score([reference[0]], word_tokenize(str(llama_result).lower()))
    print(f"2. METEOR Score: LlamaIndex ({meteor_score_value:.4f})")

    _, _, llama_bert_f1 = bert_score([str(llama_result)], [query], lang="en", verbose=False)
    bert_score_value = llama_bert_f1[0].item()
    print(f"3. BERTScore (F1): LlamaIndex ({bert_score_value:.4f})")

    try:
        relevance_model = pipeline("text-classification", model="cross-encoder/ms-marco-MiniLM-L-12-v2")
        llama_relevance = relevance_model({"text": query, "text_pair": str(llama_result)})

        relevance_score = llama_relevance[0]['score'] if isinstance(llama_relevance, list) else llama_relevance['score']
        print(f"4. Relevance Score: LlamaIndex ({relevance_score:.4f})")
    except Exception as e:
        print(f"4. Relevance Score: Error calculating relevance score - {str(e)}")
        relevance_score = None

    def hallucination_score(query, answer):
        query_words = set(query.lower().split())
        answer_words = set(answer.lower().split())
        new_words = answer_words - query_words
        return len(new_words) / len(answer_words)

    hallucination_score_value = hallucination_score(query, str(llama_result))
    print(f"5. Hallucination Score: LlamaIndex ({hallucination_score_value:.4f})")

    print("\n---\n")

    return {
        "Query": query,
        "Answer": str(llama_result),
        "Response Time": llama_time,
        "ROUGE-L F1 Score": rouge_score,
        "METEOR Score": meteor_score_value,
        "BERTScore": bert_score_value,
        "Relevance Score": relevance_score,
        "Hallucination Score": hallucination_score_value
    }

In [96]:
queries = [
    "What is tuition cost for the program?",
    "What scholarships are available for the program?",
    "What are the minimum scores for the TOEFL and IELTS English Language Requirement?",
    "Is there an application fee waiver?",
    "What are the deadlines for the in-person program?",
    "How long will it take for me to receive a decision on my application?",
    "Can I set up an advising appointment with the enrollment management team?",
    "Where can I mail my official transcripts?",
    "Does the Master’s in Applied Data Science Online program provide visa sponsorship?",
    "How do I apply to the MBA/MS program?",
    "Is the MS in Applied Data Science program STEM/OPT eligible?", 
    "How many courses must you complete to earn UChicago’s Master’s in Applied Data Science?"
]

In [81]:
# Save the result as TXT file
def save_results(results, file_format):
    file_path = f"Evaluation_Result/RAG Evaluation Result (After Fine-Tune).{file_format}"

    if file_format == 'txt':
        with open(file_path, 'w') as txtfile:
            for result in results:
                for key, value in result.items():
                    txtfile.write(f"{key}: {value}\n")
                txtfile.write("\n---\n\n")

    print(f"Results saved to {file_path}")

In [97]:
# Evaluate all queries and store results
results = []
for query in queries:
    result = evaluate_rag_systems(query)
    results.append(result)

# Save results as a text file
# save_results(results, 'txt')

2024-11-10 21:11:55,417 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


2024-11-10 21:11:55,417 - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


2024-11-10 21:11:56,235 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-11-10 21:11:56,265 - INFO - Using default tokenizer.


2024-11-10 21:11:56,235 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Query: What is tuition cost for the program?

LlamaIndex Answer: Tuition for the MS in Applied Data Science program is $5,967 per course or $71,604 total tuition.
LlamaIndex Response Time: 1.22 seconds

Evaluation:
2024-11-10 21:11:56,265 - INFO - Using default tokenizer.
1. ROUGE-L F1 Score: LlamaIndex (0.3077)
2. METEOR Score: LlamaIndex (0.4043)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


...

2024-11-10 21:12:22,208 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
Query: How many courses must you complete to earn UChicago’s Master’s in Applied Data Science?

LlamaIndex Answer: You must successfully complete 12 courses (6 core, 4 elective, 2 Capstone) and the Career Seminar.
LlamaIndex Response Time: 1.10 seconds

Evaluation:
2024-11-10 21:12:22,211 - INFO - Using default tokenizer.
1. ROUGE-L F1 Score: LlamaIndex (0.1250)
2. METEOR Score: LlamaIndex (0.1042)


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


3. BERTScore (F1): LlamaIndex (0.8492)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


4. Relevance Score: LlamaIndex (0.3136)
5. Hallucination Score: LlamaIndex (0.7500)

---

